In [1]:
using Laplacians

objc[5048]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[5048]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[5048]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[5048]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("../../../src/pcgMatNorm.jl")
include("../../../src/fastCSC.jl")
include("../../../src/KMPSolver.jl")
include("../../../src/samplingSolver.jl")

checkError (generic function with 1 method)

In [20]:
n = 100000;
a = wtedChimera(n,23); la = lap(a) + speye(n) * 1e-15;
truex = rand(n); b = la * truex;

### Trying out augTreeSolver

In [21]:
F = augTreePrecon(la, treeAlg=akpw);

In [22]:
function fat(b; tol=1e-4, maxits=Inf, maxtime=Inf, verbose=false)
    return pcg(la, b, F, tol=tol, maxits=maxits, maxtime=maxtime, verbose=verbose)
end

fat (generic function with 1 method)

In [23]:
@time fat(b, verbose=true);

PCG stopped after: 115 iterations with relative error 7.249979822692232e-5.
  1.364464 seconds (10.47 k allocations: 878.567 MB, 8.72% gc time)


In [24]:
function fatMN(b; tol=1e-4, maxits=Inf, maxtime=Inf, verbose=false)
    return pcgMatNorm(la, b, F, truex, tol=tol, maxits=maxits, maxtime=maxtime, verbose=verbose)
end

fatMN (generic function with 1 method)

In [25]:
@time fatMN(b, verbose=true);

PCG stopped after: 622 iterations with relative error 9.924057662695772e-5.
 12.235810 seconds (35.00 k allocations: 6.493 GB, 8.27% gc time)


### Trying for KMP

In [26]:
params = KMPparams(1/36, 6, 0.125, 600, :akpw);

a2 = copy(a);
tree = akpw(a2);
ord = Laplacians.dfsOrder(tree);
aord = symPermuteCSC(a2,ord);
tord = symPermuteCSC(tree,ord)
la = lap(aord);
fsub = KMPLapPrecon(aord, tord, params, verbose=true);

aveStretch : 5.378693867716532 fac : 11.167271420603699
level 0. Dimension 100000 off-tree edges : 237763
level 1. Dimension 21866 off-tree edges : 8318


In [27]:
function fkmp(b::Array{Float64,1}; tol=1e-4, maxits=Inf, verbose=false)
    bord = b[ord]

    xord = pcg(la, bord, fsub, tol=tol, maxits=maxits, verbose=verbose)

    x = zeros(Float64,n)
    x[ord] = xord
    subMean!(x)
    return x
end

fkmp (generic function with 1 method)

In [28]:
@time fkmp(b, verbose=true);

PCG stopped after: 93 iterations with relative error 7.482332382062994e-5.
  2.492771 seconds (56.83 k allocations: 1.851 GB, 10.61% gc time)


In [29]:
function fkmpMN(b::Array{Float64,1}; tol=1e-4, maxits=Inf, verbose=false)
    bord = b[ord]

    xord = pcgMatNorm(la, bord, fsub, truex[ord], tol=tol, maxits=maxits, verbose=verbose)

    x = zeros(Float64,n)
    x[ord] = xord
    subMean!(x)
    return x
end

fkmpMN (generic function with 1 method)

In [30]:
@time fkmpMN(b, verbose=true);

PCG stopped after: 261 iterations with relative error 9.926358402552189e-5.
  9.407370 seconds (148.26 k allocations: 5.971 GB, 10.24% gc time)


### Sampling Solver

In [31]:
@time F,gOp,U,d,ord,cn,cntime = buildSolver(a, eps=0.5, sampConst=0.02, beta=1e3, verbose=false);
checkError(gOp)

  2.914548 seconds (9.75 M allocations: 1009.393 MB, 18.30% gc time)


0.8888515210652563

In [32]:
la = lap(symPermuteCSC(a, ord));

In [33]:
function fss(b)
    auxb = copy(b)
    push!(auxb, -sum(auxb))

    ret = pcg(la, auxb[ord], F, tol=1e-4, maxits=Inf, maxtime=Inf, verbose=true)
    ret = ret[invperm(ord)]

    # We want to discard the nth element of ret (which corresponds to the first element in the permutation)
    ret = ret - ret[n]

    pop!(ret)

    return ret
end

fss (generic function with 1 method)

In [34]:
@time fss(b);

PCG stopped after: 128 iterations with relative error 9.514118898949973e-5.
  3.637007 seconds (25.60 M allocations: 1.150 GB, 6.63% gc time)


In [35]:
function fssMN(b)
    auxb = copy(b)
    push!(auxb, -sum(auxb))

    ret = pcgMatNorm(la, auxb[ord], F, truex[ord], tol=1e-4, maxits=Inf, maxtime=Inf, verbose=true)
    ret = ret[invperm(ord)]

    # We want to discard the nth element of ret (which corresponds to the first element in the permutation)
    ret = ret - ret[n]

    pop!(ret)

    return ret
end

fssMN (generic function with 1 method)

In [36]:
@time fssMN(b);

PCG stopped after: 234 iterations with relative error 9.721828404400597e-5.
  7.794090 seconds (46.81 M allocations: 2.799 GB, 7.54% gc time)
